# py_魔术方法

## 1 `__all__`  

### 定义：
是由string元素组成的list变量。
### 作用：
1. 导入 `from <module> import *` 时，限制仅导入 list 中的属性和方法。 可以隐藏不想被import的属性和方法。
2. 对于 `from <module> import <member>` 导入方式并没有影响，仍然可以从外部导入。

### 案例：
参考：https://blog.csdn.net/sxingming/article/details/52903377

## 2 `__slots__`

### 2.1 定义：
是由string元素组成的list变量。

### 2.2 作用：
1. 当一个类需要创建大量实例时，可以通过`__slots__`声明实例所需要的属性。
2. 可以利用slots的特性来限制实例的属性。
3. 优点：更快的属性访问速度；减少内存的消耗。
4. 使用`__slots__`后会丧失动态添加属性和弱引用的功能，即取消`__dict__`和`__weakref__`的生成。可能会引起其他错误，所以在一般情况下不要使用它。

### 2.3 `__slots__`纯python的实现
参考：https://www.cnblogs.com/rainfd/p/slots.html

In [1]:
class Member(object):
    # 定义描述器实现slots属性的查找
    def __init__(self, i):
        self.i = i
        
    def __get__(self, obj, type=None):
        return obj._slotvalues[self.i]
    
    def __set__(self, obj, value):
        obj._slotvalues[self.i] = value

In [3]:
class Type(type):
    # 使用元类实现slots
    def __new__(self, name, bases,  namespace):
        slots = namespace.get('_slots_')
        
        if slots:
            for i, slot in enumerate(slots):
                namespace[slot] = Member(i)
            original_init = namespace.get('__init__')
                                          
            def __init__(self, *args, **kwargs):
                # 创建_slotvalues列表和调用原来的__init__
                self._slotvalues = [None] * len(slots)
                                          
                if original_init(self, *args, **kwargs):
                    original_init(self, *args, **kwargs)
            namespace['__init__'] = __init__
                                          
        return type.__new__(self, name, bases, namespace)

In [4]:
class Object(object): 
    __metaclass__ = Type

In [5]:
class A(Object):
    _slots_ = 'x', 'y'

In [9]:
a = A()
a.x = 10
print(a.x)

10


In [12]:
# 验证访问速度
from timeit import repeat

class A(object): pass

class B(object): __slots__ = ('x')

def get_set_del_fn(obj):
    def get_set_del():
        obj.x = 1
        obj.x
        del obj.x
    return get_set_del

a = A()
b = B()
ta = min(repeat(get_set_del_fn(a)))
tb = min(repeat(get_set_del_fn(b)))
print("%.2f%%" % ((ta/tb - 1)*100))

23.65%


In [21]:
# 测试内存占用情况
from string import ascii_letters
from pympler.asizeof import asizesof

def slots_memory(num=0):
    attrs = list(ascii_letters[:num])
    class Unslotted(object): pass
    class Slotted(object): __slots__ = attrs
    unslotted = Unslotted()
    slotted = Slotter()
    
    for attr in attrs:
        unslotted.__dict__[attr] = 0
        exec('slotted.%s = 0' % attr, globals(), locals())
    memory_use = asizesof(slotted, unslotted, unslotted.__dict__)
    return memory_use

def slots_test(nums):
    return [slots_memory(num) for num in nums]

## 2.4 使用笔记

#### 2.4.1. 字符串赋值
整个字符串当做一个熟悉赋值

In [33]:
class A(object): __slots__ = ('a', 'b', 'c')
class B(object): __slots__ = 'abcd'

In [31]:
B.__slots__

'abcd'

In [32]:
A.__slots__

('a', 'b', 'c')

#### 2.4.2. `__slots__`继承问题
**注意**：
1. 建议最好直接继承`object`。
2. 如果需要用到其他父类，则父类和子类都要定义slots，还要记得子类的slots会覆盖父类的slots。
3. 除非所有父类的slots都为空，否则不要使用多继承。

In [39]:
## 1. 父类有，子类没有：
# 子类的实例还是会自动创建 __dict__ 来存储属性；
# 不过父类 __slots__ 已有的属性不受影响。

class Father(object): __slots__ = ('x')
class Son(Father): pass 

son = Son()
son.x, son.y = 1, 1
son.__dict__

{'y': 1}

In [40]:
## 2. 父类没有，子类有：
# 虽然子类取消了__dict__，但继承父类后它会继续生成。
# 同上面一样，__slots__已有的属性不受影响。

class Father(object): pass
class Son(Father): __slots__ = ('x')
son = Son()
son.x, son.y = 1, 1
son.__dict__

{'y': 1}

In [44]:
## 3. 父类有，子类有：
# 只有子类的__slots__有效，访问父类有子类没有的属性依然会报错。

class Father(object): __slots__ = ('x', 'y')
class Son(Father): __slots__ = ('x', 'z')
son = Son()
son.x, son.y, son.z = 1, 1, 1
# son.__dict__
son.x, son.y

(1, 1)

In [47]:
## 4. 多个拥有非空slots的父类：
# 由于__slots__的实现不是简单的列表或字典，多个父类的非空__slots__不能直接合并，
# 所以使用时会报错（即使多个父类的非空__slots__是相同的）。

class Father(object): __slots__ = ('x')
class Mother(object): __slots__ = ('x')
# class Son(Father, Mother): pass

In [48]:
## 5. 多个空slots的父类：
# 这是关于slots使用多继承唯一办法。

In [49]:
## 6. 某些父类有，某些父类没有：
# 跟第一种情况类似。

#### 2.4.3. 添加`__dict__`获取动态特性
在特殊情况下，可以在`__slots__`里添加`__dict__`来获取与普通实例同样的动态特性。

In [51]:
class A(object): __slots__ = ()
class B(A): __slots__ = ('__dict__', 'x')
b = B()
b.x, b.y = 1, 1
b.__dict__

{'y': 1}

#### 2.4.4. 添加`__weakref__`获取弱引用功能  
`__slots__`的实现不仅取消了`__dict__`的生成，也取消了`__weakref__`的生成。同样的，在`__slots__`将其添加可以重新获取弱引用这一功能。

#### 2.4.5. 不能通过类属性给实例设定默认值
定义了`__slots__`后，这个类的类属性都变为了描述器。如果给类属性赋值，就会把描述器给覆盖了。

#### 2.4.6. namedtuple
利用内置的namedtuple不可变的特性，结合slots，能创建出一个轻量不可变的实例。(约等于一个元组的大小)

In [55]:
from collections import namedtuple

class MyNt(namedtuple('MyNt', 'bar baz')): __slots__ = ()
    

nt = MyNt('r', 'z')
nt.bar, nt.baz

('r', 'z')

# 3 描述符
1. `__getattr`
2. `__getattribute__`
3. `__getitem__`
4. `__get__`
5. `__set__`


### 定义：
只要一个object attribute(对象属性)定义了上面三个方法中的任意一个，那么这个类就可以被称为描述符类。

In [1]:
class RevealAccess():
    def __get__(self, obj, objtype):
        print(f"self in RevealAccess:{self}")
        print(f"self: {self}\nobj: {obj}\nobjtype:{objtype}")
        
class Myclass():
    x = RevealAccess()
    def test(self):
        print(f"self in Myclass: {self}")

In [5]:
m = Myclass()
m.test()

self in Myclass: <__main__.Myclass object at 0x000001D7D8AB81D0>


In [3]:
m.x

self in RevealAccess:<__main__.RevealAccess object at 0x000001D7D8AA4E80>
self: <__main__.RevealAccess object at 0x000001D7D8AA4E80>
obj: <__main__.Myclass object at 0x000001D7D8AB8390>
objtype:<class '__main__.Myclass'>


## 3 `__repr__`和`__str__`

In [15]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        return f'Pair({self.x}, {self.y})'
    
    def __str__(self):
        return f'({self.x}, {self.y})'

In [16]:
p = Pair(3, 4)
p

Pair(3, 4)

In [17]:
print(p)

(3, 4)
